### Imports

In [1]:
import os
import numpy as np
import pandas as pd

### Inputs

In [2]:
radnet_input_df = pd.read_csv("../inputs/radnet_input.csv")
radnet_input_apmr_edge = radnet_input_df['apmr_edge'].values
radnet_input_mfsr_edge = radnet_input_df['mfsr_edge'].values
radnet_input_idg_global = radnet_input_df['idg_global'].values

- apmr_edge (Active Prefix Message Ratio): Taxa de eficiência dos Active Prefixes no encaminhamento de mensagens através das arestas, 
medindo quão bem os campos de prefixo conseguem realizar matching probabilístico para encaminhar mensagens entre nós vizinhos.

- mfsr_edge (Message Forwarding Success Ratio): Proporção de mensagens transmitidas com sucesso através de uma aresta específica, 
considerando interferência, congestionamento e qualidade da conexão sem fio entre dois nós conectados.

- idg_global (Interest Group Delivery): Eficácia global da entrega de mensagens no modo Interest-Group addressing, 
medindo quão bem o protocolo consegue localizar e entregar mensagens aos nós que compartilham o mesmo interesse da aplicação.

In [3]:
inference_output_df = pd.read_csv("../outputs/inference_output.csv")
inference_output_class = inference_output_df['class'].values
inference_output_confidence = inference_output_df['confidence'].values

- confidence (Confiança da Inferência do Modelo GNN): Eficácia do modelo gerado pelo GNN medido na atapa anterior.

In [4]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

### Fusion

In [5]:
# pesos podenrados
w_model, w_apmr, w_mfsr, w_idg = 0.4, 0.25, 0.25, 0.1
w_sum = w_model + w_apmr + w_mfsr + w_idg

In [6]:
confidence_model = np.where(inference_output_class == 1,
    inference_output_confidence,
    1.0 - inference_output_confidence
)

In [7]:
fused = (
    w_model * confidence_model +
    w_apmr * radnet_input_apmr_edge +
    w_mfsr * radnet_input_mfsr_edge +
    w_idg  * radnet_input_idg_global
) / w_sum

In [8]:
# classe final a partir do filtrado
cls = (fused >= 0.5).astype(int)

# confiança orientada à classe escolhida
conf = np.where(cls == 1, fused, 1.0 - fused)

### Apply

In [9]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'APMR':<6} {'MFSR':<6} {'IDG':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*70}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{radnet_input_apmr_edge[i]:<6.2f} {radnet_input_mfsr_edge[i]:<6.2f} {radnet_input_idg_global[i]:<6.2f} "
          f"{cls[i]:<8} {CLASS_NAMES[int(cls[i])]:<8} {conf[i]:<10.2f}")

RSSI   ETX   Delay  Busy   APMR   MFSR   IDG    Classe   Status   Confiança 
----------------------------------------------------------------------
-74    3.7   91     0.31   0.79   0.94   0.85   1        Bom      0.52      
-31    1.7   29     0.06   0.77   0.76   0.78   1        Bom      0.86      
-68    4.0   96     0.70   0.94   0.85   0.90   1        Bom      0.54      
-51    1.5   17     0.25   0.47   0.66   0.59   1        Bom      0.74      
-46    1.1   17     0.38   0.81   0.87   0.71   1        Bom      0.88      
-80    7.4   92     0.65   0.84   0.85   0.77   1        Bom      0.50      
-39    1.1   15     0.04   0.67   0.70   0.63   1        Bom      0.80      
-57    7.9   96     0.61   0.64   0.64   0.65   0        Ruim     0.61      
-67    3.9   57     0.25   0.79   0.84   0.68   1        Bom      0.62      
-62    8.6   83     0.43   0.57   0.67   0.50   0        Ruim     0.64      
-49    2.7   33     0.28   0.90   0.77   0.90   1        Bom      0.89      
-62  

### Save

In [10]:
fusion_output = pd.DataFrame({
    'class': cls,
    'confidence': conf
})

os.makedirs("../outputs", exist_ok=True)
fusion_output.to_csv("../outputs/fusion_output.csv", index=False)

print("Fusão de Dados salva em ../outputs/fusion_output.csv")

Fusão de Dados salva em ../outputs/fusion_output.csv
